# Analyze Results

In [ ]:
import os.path
import pandas as pd

from dbispipeline.analytics import get_results_as_dataframe
from hit_prediction_code.analytics import add_approach_to_df
from hit_prediction_code.analytics import add_cv_epoch_evaluator_outcome_to_df

results = get_results_as_dataframe(project_name='hit-prediction-ismir2020', table_name='hit_prediction')
results = results[results['date'] > '2021-02-02']

add_approach_to_df(results)
add_cv_epoch_evaluator_outcome_to_df(results)

In [ ]:
import numpy as np

from matplotlib import pyplot
import seaborn as sns

from hit_prediction_code.analytics import foreach_epoch

i = -1

for _, result in results.iterrows():
    mean_result = result['mean']
    
    cm_results = mean_result.loc['confusion_matrix']
    
    target_counts = np.sum(cm_results[0], axis=1) + 1
    divisor = np.broadcast_to(target_counts + 1, (len(target_counts), (len(target_counts)))).transpose()

    normalized_cm = []

    for cm in cm_results:
        cm = np.array(cm)
        normalized_cm.append(cm / divisor)
    
#     for i in range(19, int(len(normalized_cm)), 20):
    pyplot.subplots(figsize=(10,8))
    plot = sns.heatmap(normalized_cm[i])
    plot.set_title('%s with %s Epoches' % (result['approach'], mean_result.columns[i]))
    pyplot.show()
#     pyplot.savefig('../results/cm_norm_25_200.pdf')